# 2-D image manifold: result table

## Setup

In [1]:
import sys
import numpy as np
from itertools import product


## Load results

In [2]:
n_runs = 10

In [3]:
algo_filenames = []
algo_additionals = []
algo_labels = []
algo_dividers = []

def add_algo(filename, add, label):
    algo_filenames.append(filename)
    algo_additionals.append(add)
    algo_labels.append(label)
    
    
def add_divider():
    algo_dividers.append(len(algo_filenames))

add_algo("flow", "april", r"\af{}")
add_algo("pie", "april", r"\pie{}")
add_algo("mf", "april", r"\mf{}")
add_algo("emf", "april", r"\mfe{}")

n_algos = len(algo_filenames)


In [4]:
def load(name, shape, numpyfy=True, chains=1, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add in zip(algo_filenames, algo_additionals):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)

            try:
                this_result = np.load(
                    "{}/{}_2_gan2d_{}{}_{}.npy".format(result_dir, algo_filename, algo_add, run_str, name)
                )
                if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                    algo_results.append(this_result.reshape(shape))
                else:
                    algo_results.append(np.nan*np.ones(shape))

            except FileNotFoundError as e:
                # print(e)
                if shape is None:
                    algo_results.append(None)
                else:
                    algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    if numpyfy:
        all_results = np.array(all_results, dtype=np.float)
        all_results = all_results.reshape([all_results.shape[0], n_runs] + list(shape))
        
    return all_results


model_test_reco_xs = load("model_x_reco_test", (100, 3, 64, 64))
model_test_reco_errors = load("model_reco_error_test", (100,))
model_gen_fids = load("samples_fid", (1,)).squeeze()
model_gen_fids.shape


(4, 10)

In [5]:
max_reco_error = 10000.
model_mean_reco_errors = np.mean(np.clip(model_test_reco_errors, 0., max_reco_error), axis=2)
model_mean_reco_errors.shape


(4, 10)

## Outlier removal

In [6]:
def mean_err_without_outliers(data, remove=1):
    shape = list(data.shape)[:-1]
    data.reshape((-1, data.shape[-1]))
    
    means, errors = [], []
    
    for data_ in data:
        data_ = data_[np.isfinite(data_)]
        if not len(data_) > 0:
            means.append(np.nan)
            errors.append(np.nan)
            continue
            
        if len(data_) > 2*remove + 1:
            for _ in range(remove):
                data_ = np.delete(data_, np.argmin(data_))
                data_ = np.delete(data_, np.argmax(data_))

        means.append(np.mean(data_))
        errors.append(np.std(data_) / len(data_)**0.5)
        
    return np.array(means).reshape(shape), np.array(errors).reshape(shape)
    
    
model_fid_mean, model_fid_std = mean_err_without_outliers(model_gen_fids)
model_reco_error_mean, model_reco_error_std = mean_err_without_outliers(model_mean_reco_errors)


## Best metrics

In [7]:
best_fid = -1
best_reco = -1

best_fid = np.nanargmin(model_fid_mean)
print(algo_labels[best_fid])

best_reco = np.nanargmin(np.where(model_reco_error_mean > 1., model_reco_error_mean, np.nan))
print(algo_labels[best_reco])


\mfe{}
\mfe{}


## Print result table

In [8]:
def print_results(
    l_label=max([len(l) for l in algo_labels]), l_mean=5, l_err=3, latex=False, after_decs=(1,0), labels=["FID", "Reco error"]
):
    # Number of digits
    l_result = l_mean + 2 + l_err
    l_total = l_label + 1 + 2 * (3 + l_result)
        
    # Divider
    empty_result = "" if latex else " "*(l_result + 1)
    col_divider = "&" if latex else "|"
    line_end = r"\\" if latex else ""
    block_divider = r"\midrule" if latex else "-"*l_total
    
    # Number formatting
    def _f(val, err, after_dec, best=False):
        if not np.any(np.isfinite(val)):
            return empty_result
        
        result = "{:>{}.{}f}".format(val, l_mean, after_dec)
        if latex and best:
            result = r"\textbf{" + result + "}"
            
        if latex:
            err_str = str.rjust("({:.{}f})".format(err, after_dec), l_err + 2).replace(" ", r"\hphantom{0}")
            result += r"\,\textcolor{darkgray}{" + err_str + "}"
        else:
            err_str = "({:>{}.{}f})".format(err, l_err, after_dec)
            result += err_str
            
        result += "*" if not latex and best else " "
        
        if latex:
            result = result.replace("-", "$-{}$")
            result = result.replace("darkgray", "dark-gray")
        return result
    
    
    # Header
    print(f"{'':<{l_label}.{l_label}s} {col_divider} {labels[0]:>{l_result}.{l_result}s} {col_divider} {labels[1]:>{l_result}.{l_result}s} {line_end}")
    print(block_divider)

    # Iterate over methods
    for i, (label, fid, fid_err, reco, reco_err) in enumerate(zip(
        algo_labels, model_fid_mean, model_fid_std, model_reco_error_mean, model_reco_error_std
    )):
        # Divider
        if i in algo_dividers:
            print(block_divider)
            
        # Print results
        print(
            f"{label:<{l_label}.{l_label}s} {col_divider} "
            + f"{_f(fid, fid_err, after_decs[0], i==best_fid):s}{col_divider} "
            + f"{_f(reco, reco_err, after_decs[1], i==best_reco):s} {line_end}"
        )


In [9]:
print_results()

       |        FID | Reco error 
---------------------------------
\af{}  |  57.8(0.8) |     0(  0)  
\pie{} | 140.5(3.2) |  5554( 50)  
\mf{}  |  43.9(0.1) |   330(  8)  
\mfe{} |  43.5(0.1)*|   301(  2)* 


In [10]:
print_results(latex=True)

       &        FID & Reco error \\
\midrule
\af{}  &  57.8\,\textcolor{dark-gray}{(0.8)} &     0\,\textcolor{dark-gray}{\hphantom{0}\hphantom{0}(0)}  \\
\pie{} & 140.5\,\textcolor{dark-gray}{(3.2)} &  5554\,\textcolor{dark-gray}{\hphantom{0}(50)}  \\
\mf{}  &  43.9\,\textcolor{dark-gray}{(0.1)} &   330\,\textcolor{dark-gray}{\hphantom{0}\hphantom{0}(8)}  \\
\mfe{} & \textbf{ 43.5}\,\textcolor{dark-gray}{(0.1)} & \textbf{  301}\,\textcolor{dark-gray}{\hphantom{0}\hphantom{0}(2)}  \\


In [11]:
model_gen_fids

array([[ 58.20174533,  54.84207584,  54.31252978,  68.97241479,
         50.90466642,  59.34929282,  59.31889101,  57.69975286,
         57.2841141 ,  61.74553937],
       [152.73049773, 136.81524045, 140.40260621, 165.56874879,
        133.06243176, 150.22100708, 105.30221772, 150.00044463,
        125.05236849, 136.04376506],
       [ 44.33512519,  44.82221643,  43.70995254,  44.43353831,
         44.3712352 ,  43.4974198 ,  43.69989727,  43.31443439,
         43.27895678,  43.58872495],
       [ 43.45379455,  43.74331072,  42.8844404 ,  43.49512616,
         42.3594556 ,  44.55934332,  44.03446856,  43.26561651,
         43.48677179,  43.74975561]])